In [4]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt comet_ml

import torch
import utils
display = utils.notebook_init()

YOLOv5  v7.0-287-g574331f9 Python-3.10.13 torch-2.2.0+cpu CPU


Setup complete  (8 CPUs, 7.4 GB RAM, 135.5/475.8 GB disk)


In [5]:
%cd yolov5

[WinError 2] The system cannot find the file specified: 'yolov5'
C:\Users\bjkk3\OneDrive\Desktop\FishClassification\YOLO_Classification\yolov5


In [2]:
pwd

'C:\\Users\\bjkk3\\OneDrive\\Desktop\\FishClassification\\YOLO_Classification'

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 16 --epochs 80 --data custom_data.yaml --weights yolov5s.pt --cache
!python train.py --img 640 --batch 16 --epochs 80 --data custom_data.yaml --weights yolov5n.pt --cache

- Exp 8 has weights for extra-large model
- Exp 9 has weights for small model
- Exp 10 has weights for nano model

In [ ]:
import cv2
import torch
import os

# Correct paths to the model weights
model_path_exp8 = '/content/drive/MyDrive/FishClassification/YOLOV5FishClassification/yolov5/runs/train/exp8/weights/best.pt'
model_path_exp10 = '/content/drive/MyDrive/FishClassification/YOLOV5FishClassification/yolov5/runs/train/exp10/weights/best.pt'

# Load the models
xl_model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path_exp8, force_reload=True)
nano_model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path_exp10, force_reload=True)

# Directory containing videos
videos_dir = '/content/drive/MyDrive/FishClassification/laxaleir2022_vid'

# Directory to save confirmed frames
save_dir = '/content/drive/MyDrive/FishClassification/confirmed_frames'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

def process_video(video_path, nano_model, xl_model, save_dir):
    cap = cv2.VideoCapture(video_path)
    fish_detected = False

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        # Initial detection with nano model
        results = nano_model(frame_rgb)

        # Check for detections of class 0 (fish) specifically
        for *xyxy, conf, cls in results.xyxy[0]:
            if cls == 0:  # If a fish is detected
                # Confirmation with the extra-large model
                xl_results = xl_model(frame_rgb)

                # Check for confirmations of class 0 (fish) specifically in the XL model results
                for *xyxy, conf, cls in xl_results.xyxy[0]:
                    if cls == 0:  # If a fish is confirmed
                        fish_detected = True
                        # Generate a filename for the saved frame
                        frame_filename = f"confirmed_fish_{os.path.basename(video_path).split('.')[0]}.jpg"
                        # Save the frame
                        save_path = os.path.join(save_dir, frame_filename)
                        cv2.imwrite(save_path, cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR))  # Convert RGB back to BGR for saving
                        print(f"Fish confirmed and frame saved to: {save_path}")
                        return fish_detected  # Stop after the first confirmation and save

    cap.release()
    return fish_detected

# Process every 50th video in the directory
video_files = sorted([f for f in os.listdir(videos_dir) if f.endswith('.mp4')])
for i, video_file in enumerate(video_files):
    if i % 50 == 0:  # process every 50th video file
        video_path = os.path.join(videos_dir, video_file)
        print(f"Processing video: {video_path}")
        if process_video(video_path, nano_model, xl_model, save_dir):
            print(f"Fish detection confirmed in video: {video_path}")
        else:
            print(f"No fish detection confirmed in video: {video_path}")


In [ ]:
import time

def process_video(video_path, nano_model, xl_model, save_dir):
    cap = cv2.VideoCapture(video_path)
    fish_detected = False

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        # Initial detection with nano model
        results = nano_model(frame_rgb)

        # Check for detections of class 0 (fish) specifically
        for *xyxy, conf, cls in results.xyxy[0]:
            if cls == 0:  # If a fish is detected
                # Confirmation with the extra-large model
                xl_results = xl_model(frame_rgb)

                # Check for confirmations of class 0 (fish) specifically in the XL model results
                for *xyxy, conf, cls in xl_results.xyxy[0]:
                    if cls == 0:  # If a fish is confirmed
                        fish_detected = True
                        frame_filename = f"confirmed_fish_{os.path.basename(video_path).split('.')[0]}.jpg"
                        save_path = os.path.join(save_dir, frame_filename)
                        cv2.imwrite(save_path, cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR))
                        print(f"Fish confirmed and frame saved to: {save_path}")
                        return fish_detected

    cap.release()
    return fish_detected

# Process every 50th video and measure the time taken for each
video_files = sorted([f for f in os.listdir(videos_dir) if f.endswith('.mp4')])
for i, video_file in enumerate(video_files):
    if i % 50 == 0:  # process every 50th video file
        video_path = os.path.join(videos_dir, video_file)
        print(f"Processing video: {video_path}")

        start_time = time.time()  # Start the stopwatch before processing the video
        if process_video(video_path, nano_model, xl_model, save_dir):
            print(f"Fish detection confirmed in video: {video_path}")
        else:
            print(f"No fish detection confirmed in video: {video_path}")
        end_time = time.time()  # Stop the stopwatch after processing the video

        print(f"Time taken for processing video {video_file}: {end_time - start_time:.2f} seconds")


Processing video: /content/drive/MyDrive/FishClassification/laxaleir2022_vid/1.mp4
No fish detection confirmed in video: /content/drive/MyDrive/FishClassification/laxaleir2022_vid/1.mp4
Time taken for processing video 1.mp4: 4.51 seconds
Processing video: /content/drive/MyDrive/FishClassification/laxaleir2022_vid/1057.mp4
Fish confirmed and frame saved to: /content/drive/MyDrive/FishClassification/confirmed_frames/confirmed_fish_1057.jpg
Fish detection confirmed in video: /content/drive/MyDrive/FishClassification/laxaleir2022_vid/1057.mp4
Time taken for processing video 1057.mp4: 0.09 seconds
Processing video: /content/drive/MyDrive/FishClassification/laxaleir2022_vid/1114.mp4
Fish confirmed and frame saved to: /content/drive/MyDrive/FishClassification/confirmed_frames/confirmed_fish_1114.jpg
Fish detection confirmed in video: /content/drive/MyDrive/FishClassification/laxaleir2022_vid/1114.mp4
Time taken for processing video 1114.mp4: 0.09 seconds
Processing video: /content/drive/MyDri

In [ ]:
!python detect.py --weights runs/train/exp8/weights/best.pt --img 640 --conf 0.25 --source ../43.mp4


In [ ]:
!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.225.183.233:8501



In [ ]:
import os
import cv2
import torch
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, cohen_kappa_score

checkpoint_path = 'runs/train/exp8/weights/best.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=checkpoint_path, force_reload=True)  # force_reload to ensure latest checkpoint is loaded

videos_folder = '../test/'

all_ground_truths = []  # This should be filled with actual ground truth data
all_predictions = []

for video_file in os.listdir(videos_folder):
    if video_file.endswith('.mp4'):  # Ensure the file is an MP4 video
        video_path = os.path.join(videos_folder, video_file)
        cap = cv2.VideoCapture(video_path)

        video_predictions = []  # Store predictions for each frame

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Inference
            results = model(frame)

            pred_labels = results.xyxy[0][:, -1].cpu().numpy()
            pred_boxes = results.xyxy[0][:, :-1].cpu().numpy()
            confidences = results.xyxy[0][:, 4].cpu().numpy()
            prediction = int((confidences > 0.5).any())  # Assuming threshold of 0.5
            video_predictions.append(prediction)

        all_predictions.extend(video_predictions)
        cap.release()

all_ground_truths = [0,1,1,1,1,1,0,0,0,0,1,1,1,0]
all_ground_truths = np.array(all_ground_truths)
all_predictions = np.array(all_predictions)

precision, recall, f1_score,  = precision_recall_fscore_support(all_ground_truths, all_predictions, average='binary')
accuracy = np.mean(all_ground_truths == all_predictions)
conf_matrix = confusion_matrix(all_ground_truths, all_predictions)
kappa_score = cohen_kappa_score(all_ground_truths, all_predictions)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Cohens Kappa: {kappa_score}')

In [ ]:
import streamlit as st
import cv2
import numpy as np
import os
import torch


model_path_exp8 = '/content/drive/Othercomputers/My Computer/FishClassification/YOLOV5FishClassification/yolov5/runs/train/exp8/weights/best.pt'
model_path_exp10 = '/content/drive/Othercomputers/My Computer/FishClassification/YOLOV5FishClassification/yolov5/runs/train/exp10/weights/best.pt'

# Load the models
xl_model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path_exp8, force_reload=True)
nano_model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path_exp10, force_reload=True)

# Function to process video and detect fish
def process_video(video_file, nano_model, xl_model, save_dir):
    cap = cv2.VideoCapture(video_file)
    frame_number = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        # Initial detection with nano model
        results = nano_model(frame_rgb)

        # Check for detections of class 0 (fish) specifically
        for *xyxy, conf, cls in results.xyxy[0]:
            if cls == 0:  # If a fish is detected
                # Confirmation with the extra-large model
                xl_results = xl_model(frame_rgb)

                # Check for confirmations of class 0 (fish) specifically in the XL model results
                for *xyxy, conf, cls in xl_results.xyxy[0]:
                    if cls == 0:  # If a fish is confirmed
                        # Save the frame
                        frame_filename = f"confirmed_fish_frame_{frame_number}.jpg"
                        save_path = os.path.join(save_dir, frame_filename)
                        cv2.imwrite(save_path, cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR))  # Convert RGB back to BGR for saving
                        return True, frame_number  # Return True if fish is found and frame number
        frame_number += 1

    cap.release()
    return False, None

# Main function to run the Streamlit web app
def main():
    st.title("Fish Detection Web App")

    # Sidebar for uploading video
    uploaded_file = st.sidebar.file_uploader("Upload video file", type=["mp4", "avi"])

    if uploaded_file is not None:
        st.sidebar.write("Uploaded video:")
        st.sidebar.video(uploaded_file)

        # Button to start processing
        if st.sidebar.button("Start Processing"):
            save_dir = "confirmed_frames"
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)

            # Process video and detect fish
            fish_detected, frame_number = process_video(uploaded_file, nano_model, xl_model, save_dir)

            if fish_detected:
                st.subheader("Fish Detected!")
                st.write(f"Fish was detected in frame number: {frame_number}")
                st.image(os.path.join(save_dir, f"confirmed_fish_frame_{frame_number}.jpg"))
            else:
                st.subheader("Fish Not Detected :(")

if __name__ == "__main__":
    main()


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 v7.0-284-g95ebf68f Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 322 layers, 86180143 parameters, 0 gradients, 203.8 GFLOPs
Adding AutoShape... 
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 v7.0-284-g95ebf68f Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 1761871 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 
2024-02-29 08:50:04.644 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
